In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import torch.nn.functional as F
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
#import pretrainedmodels
import os
from torch.utils.data import Dataset
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import numpy as np
import pandas as pd
import os
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models



PyTorch Version:  1.5.0+cu101
Torchvision Version:  0.6.0+cu101


In [ ]:
print('Train set:')
for cls in os.listdir("../input/ammi-2020-convnets/train/train"):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join("../input/ammi-2020-convnets/train/train", cls)))))
im = Image.open('../input/ammi-2020-convnets/train/train/cgm/train-cgm-738.jpg')
print(im.size)

In [ ]:
data_path = "../input/ammi-2020-convnets/train/train"
test_path = "../input/ammi-2020-convnets/test/test"
extraimage_path = "../input/ammi-2020-convnets/extraimages/extraimages"

# Distribution of the classes in the initial dataset

In [ ]:
print('Train set:')
class_distrbution = {}
for cls in os.listdir(data_path):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join(data_path, cls)))))
    class_distrbution[cls] =  len(os.listdir(os.path.join(data_path, cls)))
im = Image.open(data_path+'/cgm/train-cgm-738.jpg')
print(im.size)
class_distrbution

In [ ]:
# Transformations for both the training and testing data

# Transformations for both the training and testing data
mean=[0.4543, 0.5137, 0.3240]
std=[0.1949, 0.1977, 0.1661]


train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224), #448, 299, 224, 331
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean,std=std)])

test_transforms = transforms.Compose([ transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])

In [ ]:


class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        class_names = {}
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])

                name = str(i)+'-'+className
                if name not in class_names:
                    class_names[name] = 1
                else:
                    class_names[name] += 1
        self.file_list = files
        print(class_names)
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
        
# #         return im.view(3, 448, 448), classCategory
        return im.view(3, 224, 224), classCategory
# #         return im.view(3, 299, 299), classCategory

In [ ]:
train_data = CassavaDataset(data_path, transform=train_transforms)

test_data = CassavaDataset(test_path, transform=test_transforms)

extraimage_data = CassavaDataset(extraimage_path, transform=train_transforms) #maybe need an other trasforms, I had to change the dataset structure :)

# Over sample minority class and under sample majority class

In [ ]:
# Creating PT data samplers and loaders:
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
#-----------------------------------------------------------------------------z

validation_split = 0.2
shuffle_dataset = True
random_seed= 42

ipdb.set_trace()
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]


# get target

targets=[]
#data=[]

for i in train_data.file_list:
    targets.append(i[0])
   # data.append(i[2])
    
target_train=targets[split:]
train_set=data[split:]
print(len(train_set))

target_test=targets[:split]
#test_set=data[:split]



    
# for i in test_data.file_list:
#     test_targets.append(i[0])
    
# target = torch.cat((torch.zeros(int(len(train_data) * 0.99), dtype=torch.long),
#                     torch.ones(int(len(train_data) * 0.01), dtype=torch.long)))


#count classes
class_count = np.unique(target_train ,return_counts=True)[1]
#print(class_count)
class_count_test= np.unique(target_test, return_counts=True)[1]


#--------------------------------------------------------------------
# get weights train

weight_train = 1. / class_count
#print(targets)
samples_weight_train = weight_train[target_train]
#print(samples_weight_train )

samples_weight_train = torch.from_numpy(samples_weight_train)
#print(samples_weight_train)
sampler_train = WeightedRandomSampler(samples_weight_train, len(samples_weight_train))

#### valid



weight_test = 1. / class_count_test
#print(targets)
samples_weight_test = weight_test[target_test]
#print(samples_weight_train )

samples_weight_test = torch.from_numpy(samples_weight_test)
#print(samples_weight_train)
sampler_valid = WeightedRandomSampler(samples_weight_test, len(samples_weight_test))


train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=sampler_train)


valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=sampler_valid)
# valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
#                                              sampler=valid_sampler)

#test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
#     img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
class Model(nn.Module):
    
    def __init__(self,input,n_class):
        super(Model,self).__init__()
        
        self.conv=nn.Sequential(nn.Conv2d(input,32,5),
                                nn.ReLU(),
#                                 nn.BatchNorm2d(32),
                                nn.MaxPool2d(3),
                                nn.Conv2d(32,64,3),
                                nn.ReLU(),
#                                 nn.BatchNorm2d(64),
                                nn.MaxPool2d(3),
                                nn.Conv2d(64,128,3),
                                nn.ReLU(),
#                                 nn.BatchNorm2d(128),
                                nn.MaxPool2d(3)
                               )
        
        self.fc1=nn.Linear(7*7*128,512)
        self.dropout1=nn.Dropout(0.2)
        
        self.fc2=nn.Linear(512,1024)
        self.dropout2=nn.Dropout(0.2)
        
        self.fc3=nn.Linear(1024,256)
        self.dropout3=nn.Dropout(0.2)
        
        self.fc4=nn.Linear(256,n_class)
        
        
        
    def forward(self,x):
        x=self.conv(x)
        
#         print('conv',x)
        x=x.view(x.size(0),-1)

        x=self.fc1(x)
        x=self.dropout1(F.relu(x))
        x=self.fc2(x)
        x=self.dropout2(F.relu(x))
#         print('fc',x)
        x=self.fc3(x)
        x=self.dropout3(F.relu(x))
        x=self.fc4(x)
        
#         print('out',x)
        
        
        return x

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

 # Model whith cross entropy loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model(3,5)
model=model.to(device)
lr = 1e-4  # 0.001

#--------------------------------------------------------------------------------------#
train_loader.dataset.classes,
class_weights = [class_distrbution[i] for i in train_loader.dataset.classes]
class_weights_normalized = [max(class_weights)/i for i in class_weights]

class_weights_normalized,torch.Tensor(class_weights_normalized)

x = torch.Tensor(class_weights_normalized)
x = x.to(device)
x = x
x,class_distrbution

#--------------------------------------------------------------------------------------#

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

num_epochs=30
total_loss_train, total_acc_train = [],[]



#---------------------------------------------------------------------------------------------#

#---------------------------------------------------------------------------------------------#


# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    n=0
    loss_list = []
    acc_list = []   
    for i, data in enumerate(train_loader):
        
#         correct=0
        
        optimizer.zero_grad()
        # Run the forward pass
        images, labels = data
        #print(images.size())

        images = Variable(images).to(device)
        labels = Variable(labels).to(device)   
        
        outputs = model(images)
        #print(outputs.size())
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        #print(total)
        prediction = outputs.max(1, keepdim=True)[1]
       
        
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/total)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print("----------------------------------------------------------------")
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
            print("----------------------------------------------------------------")

            
#######################################################################################

model.eval()
val_loss = AverageMeter()
val_acc = AverageMeter()


with torch.no_grad():
        for i, data in enumerate(valid_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')

            
            
    
    
            

#  Model with focal loss

In [ ]:
def focalLoss( outputs,targets,w,alpha=1,gamma=4):
    ce_loss = torch.nn.functional.cross_entropy(outputs, targets, reduction='none') # important to add reduction='none' to keep per-batch-item loss
    pt = torch.exp(-ce_loss)
    focal_loss = (alpha * (1-pt)**gamma * ce_loss).mean() # mean over the batch
    return focal_loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#--------------------------------------------------------------------------------------#
train_loader.dataset.classes,
class_weights = [class_distrbution[i] for i in train_loader.dataset.classes]
class_weights_normalized = [max(class_weights)/i for i in class_weights]

class_weights_normalized,torch.Tensor(class_weights_normalized)

x = torch.Tensor(class_weights_normalized)
x = x.to(device)
x = x
x,class_distrbution

#--------------------------------------------------------------------------------------#

model = Model(3,5)

model=model.to(device)

lr = 1e-4   # 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

num_epochs=30
total_loss_train, total_acc_train = [],[]


# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
   
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)

    model.train()
    n=0
    loss_list = []
    acc_list = []  
     
    for i, data in enumerate(train_loader):
        
#         correct=0
        
        optimizer.zero_grad()
        # Run the forward pass
        images, labels = data
        #print(images.size())

        images = Variable(images).to(device)
        labels = Variable(labels).to(device)   
        
        outputs = model(images)
        #print(outputs.size())
        loss =  focalLoss(outputs, labels,x)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        #print(total)
        prediction = outputs.max(1, keepdim=True)[1]
       
        
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/total)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)


#########################################################################

 model.eval()

  with torch.no_grad():
        for i, data in enumerate(valid_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(focalLoss(outputs, labels,x).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')



            